In [1]:
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()
tf.disable_v2_behavior()

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
tf.__version__

'2.4.0'

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"

print(tf.test.is_gpu_available())

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
False


In [4]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [5]:
batch_size = 128

In [6]:
x_train = x_train / 255
x_test = x_test / 255

In [7]:
n_train_batch = x_train.shape[0] // batch_size

In [8]:
x = tf.placeholder(tf.float32, [None, 784], name='x')
y = tf.placeholder(tf.float32, [None, 10], name='y')

In [9]:
w_0 = tf.get_variable('w_0', [784, 400])
b_0 = tf.get_variable('b_0', [400])

w_1 = tf.get_variable('w_1', [400, 100])
b_1 = tf.get_variable('b_1', [100])

w_2 = tf.get_variable('w_2', [100, 10])
b_2 = tf.get_variable('b_2', [10])

In [10]:
out_0 = tf.nn.relu(tf.matmul(x, w_0) + b_0)
out_1 = tf.nn.relu(tf.matmul(out_0, w_1) + b_1)
prediction = tf.matmul(out_1, w_2) + b_2

In [11]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [12]:
train_step = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [13]:
correct_prediction =  tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
epoch = 5

In [15]:
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
 
config.gpu_options.allow_growth = True 

In [16]:
sess = tf.InteractiveSession(config=config)
initializer = tf.global_variables_initializer()
sess.run(initializer)

Device mapping: no known devices.


In [17]:
for e in range(epoch):
    for batch in range(n_train_batch):

        start_index = batch * batch_size
        end_index = (batch + 1) * batch_size

        batch_x, batch_y = x_train[start_index:end_index].reshape(batch_size, 784), tf.one_hot(y_train[start_index:end_index], 10).eval(session=sess)

        sess.run([train_step, loss], feed_dict={x : batch_x, y : batch_y})
    acc = sess.run(accuracy, feed_dict = {x : x_test.reshape(10000, 784), y : tf.one_hot(y_test, 10).eval(session=sess)})
    print("iter: ", e + 1, "test acc: ", acc)

iter:  1 test acc:  0.9525
iter:  2 test acc:  0.967
iter:  3 test acc:  0.9713
iter:  4 test acc:  0.9729
iter:  5 test acc:  0.9769
